In [129]:
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import wordnet as wn
import nltk
from nltk.corpus import stopwords
import mysql.connector
import string
import os
import datetime
import numpy as np
import json
import re
import enchant
import spacy
import math

In [130]:
#connector to 2.009 db
mydb = mysql.connector.connect(user='root',
                               password='SharonF1.',
                               host='localhost',
                               port='3306',
                               database='slackdatabase',
                               auth_plugin='mysql_native_password',
                               raw=False,
                               charset='utf8'
                               )

c = mydb.cursor(buffered=True)


In [138]:
#connector to viaSep db
mydb_via = mysql.connector.connect(user='root',
                                password = 'SharonF1.',
                                host='localhost',
                                port = '3306',
                                database = 'sys',
                                #auth_plugin='mysql_native_password',
                                #raw = False
    )
c_via = mydb_via.cursor(buffered=True)

In [139]:
#pull all of the messages from 2.009, we should only need message, user, channel and timestamp for this algorithm, so we can just pull directly from the messages table

query = "SELECT * FROM slackdatabase.message JOIN slackdatabase.user ON slack_uid = id_user WHERE consent = 'Y';"
c.execute(query)
df = pd.DataFrame(c.fetchall())
df.columns = c.column_names

df.head(25)

,id,id_user,id_channel,year,id_team,comment,timestamp,reacts,replies,reply_count,...,real_name,id_team,role,is_staff,admin,bot,gender,ethnicity,first_gen,consent
0,294172,U02DULV1YG7,C02E61HR8TX,2021,39,If anyone sees an ultrium brass rat on killian...,2021-10-08 15:39:08,1,0.0,0.0,...,Catalina Romero,40,Story Officer,0.0,0.0,0.0,W,Hispanic or Latino,1.0,Y
1,294422,U02DULV1YG7,C02E61HR8TX,2021,39,Me too sorry :confused:,2021-11-08 18:19:38,0,0.0,0.0,...,Catalina Romero,40,Story Officer,0.0,0.0,0.0,W,Hispanic or Latino,1.0,Y
2,298765,U02DULV1YG7,C02JFDE38KG,2021,40,We cute,2021-12-07 13:28:25,1,0.0,0.0,...,Catalina Romero,40,Story Officer,0.0,0.0,0.0,W,Hispanic or Latino,1.0,Y
3,298766,U02DULV1YG7,C02JFDE38KG,2021,40,"And a lotta love for our presenters, fr y'all ...",2021-12-07 13:33:26,1,0.0,0.0,...,Catalina Romero,40,Story Officer,0.0,0.0,0.0,W,Hispanic or Latino,1.0,Y
4,298773,U02DULV1YG7,C02JFDE38KG,2021,40,:money_mouth_face::money_mouth_face: LESGOOOOO...,2021-10-21 17:21:16,1,0.0,0.0,...,Catalina Romero,40,Story Officer,0.0,0.0,0.0,W,Hispanic or Latino,1.0,Y
5,298776,U02DULV1YG7,C02JFDE38KG,2021,40,,2021-10-21 19:00:00,1,0.0,0.0,...,Catalina Romero,40,Story Officer,0.0,0.0,0.0,W,Hispanic or Latino,1.0,Y
6,298777,U02DULV1YG7,C02JFDE38KG,2021,40,What we need instead of Pom poms,2021-10-21 19:00:07,0,0.0,0.0,...,Catalina Romero,40,Story Officer,0.0,0.0,0.0,W,Hispanic or Latino,1.0,Y
7,298778,U02DULV1YG7,C02JFDE38KG,2021,40,Look at how sexy our product looks :hot_face:,2021-12-06 14:03:23,1,0.0,0.0,...,Catalina Romero,40,Story Officer,0.0,0.0,0.0,W,Hispanic or Latino,1.0,Y
8,298781,U02DULV1YG7,C02JFDE38KG,2021,40,Yes I did!,2021-12-06 14:39:52,0,0.0,0.0,...,Catalina Romero,40,Story Officer,0.0,0.0,0.0,W,Hispanic or Latino,1.0,Y
9,298799,U02DULV1YG7,C02JFDE38KG,2021,40,Need to grab my stuff,2021-12-06 23:19:55,0,0.0,0.0,...,Catalina Romero,40,Story Officer,0.0,0.0,0.0,W,Hispanic or Latino,1.0,Y


In [140]:
#drop duplicate columns
df = df.loc[:,~df.columns.duplicated()].copy()

In [142]:
#make a small testing df
df = df.sort_values(by=['id_channel', 'timestamp'], ascending=[True, True])
df_test = df[df['year']==2021]

In [143]:
df_test

,id,id_user,id_channel,year,id_team,comment,timestamp,reacts,replies,reply_count,...,name,real_name,role,is_staff,admin,bot,gender,ethnicity,first_gen,consent
68741,370021,U02EZ29LH0Q,C02DPVD3N79,2021,46,"I made an extremely yellow slack theme, colors...",2021-09-15 18:16:16,0,0.0,0.0,...,adapoz,Annemarie Dapoz,System Integrator,0.0,0.0,0.0,W,White,0.0,Y
68742,370022,U02EZ29LH0Q,C02DPVD3N79,2021,46,"#FFF1A6,#350D36,#FFDF0F,#2B2929,#FFF8CF,#5E5B5...",2021-09-15 18:16:17,1,1.0,1.0,...,adapoz,Annemarie Dapoz,System Integrator,0.0,0.0,0.0,W,White,0.0,Y
39781,369992,U02E9D9Q526,C02DPVD3N79,2021,46,:partying_face::partying_face:,2021-09-16 11:49:47,1,0.0,0.0,...,jhess22,Jay Hesslink,Safety Officer,0.0,0.0,0.0,M,White,0.0,Y
68735,369995,U02EZ29LH0Q,C02DPVD3N79,2021,46,<#C02F839B8H3|memes> now exists courtesy of th...,2021-09-22 18:21:43,0,0.0,0.0,...,adapoz,Annemarie Dapoz,System Integrator,0.0,0.0,0.0,W,White,0.0,Y
68744,370038,U02EZ29LH0Q,C02DPVD3N79,2021,46,is anyone able to obtain a long length of pvc ...,2021-10-08 10:11:13,0,1.0,3.0,...,adapoz,Annemarie Dapoz,System Integrator,0.0,0.0,0.0,W,White,0.0,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69238,367156,U02EZ29P5H6,C02PL83P7Q8,2021,46,Is it possible to do q&amp;a practice closer t...,2021-12-06 14:39:41,0,1.0,2.0,...,emyuan,Emily Yuan,Story Officer,0.0,0.0,0.0,W,Asian,0.0,Y
41895,367157,U02E9DA743C,C02PL83P7Q8,2021,46,So the reason it's at 5pm is so people can eat...,2021-12-06 15:05:29,0,0.0,0.0,...,svangara,Sreya Vangara,Story Officer,0.0,0.0,0.0,None,None,NaN,Y
69239,367158,U02EZ29P5H6,C02PL83P7Q8,2021,46,Ya I was thinking it would be nicer to just co...,2021-12-06 15:06:27,0,0.0,0.0,...,emyuan,Emily Yuan,Story Officer,0.0,0.0,0.0,W,Asian,0.0,Y
41896,367160,U02E9DA743C,C02PL83P7Q8,2021,46,<@U02E9D9K5DG> we are trying to figure out a p...,2021-12-06 16:19:03,0,0.0,0.0,...,svangara,Sreya Vangara,Story Officer,0.0,0.0,0.0,None,None,NaN,Y


In [146]:
#data cleaning


def filter_non_dict_words(text):
    #check spelling of words 
    d = enchant.Dict("en_US")
    # Split the text into words
    words = text.split()
    # Filter words that are not in the dictionary
    filtered_words = [word for word in words if d.check(word)]
    # Join the filtered words back into a string
    return ' '.join(filtered_words)


def generalPreprocessing(data):
    #take out @'s and <channels!>

    data['comment'] = data['comment'].map(lambda x: re.sub(r'\<(.*[^>])>',r"", x))

    #normally we would remove punctuation and stopwords, but we need these for LSM, can uncomment out if needed later

    # #remove punctuation
    # translator = str.maketrans({key: None for key in string.punctuation +"123456789-"})
    # data['comments_processed'] = data['comments_processed'].map(lambda x: x.translate(translator)) #takes out punctuation

    #tokenize words
    # data_words = list(sentence_to_words(documents))
    
    # data_words = remove_stopwords(data_words)

    #lowercase
    data['comment'] = data['comment'].map(lambda x: x.lower())
    #data['date'] = data['timestamp'].map(lambda x: x.date())

   

    data['comment'] = data['comment'].apply(filter_non_dict_words)

    #do not lemmatize for LSM but do for the LSA

    #lemmatize and keep only nouns, verbs, adjectives and adverbs
    # nlp = spacy.load('en', disable=['parser', 'ner'])
    # data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    # print("after lemmatizing")
    # print(data_lemmatized)



    return data

In [148]:
#clean the data

df_test = generalPreprocessing(df_test)

/var/folders/c3/444sznkj1hsg3mnzfrpclpj80000gn/T/ipykernel_12009/141510827.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['comment'] = data['comment'].map(lambda x: re.sub(r'\<(.*[^>])>',r"", x))
/var/folders/c3/444sznkj1hsg3mnzfrpclpj80000gn/T/ipykernel_12009/141510827.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['comment'] = data['comment'].map(lambda x: x.lower())
/var/folders/c3/444sznkj1hsg3mnzfrpclpj80000gn/T/ipykernel_12009/141510827.py:37: SettingWithCopyWarning: 
A value is 

In [149]:
#set the global values for aggregation -- based on number of messages, and time periods?

agg_num_messages = 10
agg_time = datetime.timedelta(minutes=30)
agg_date = datetime.timedelta(days=1)


In [150]:
#first need to aggregate a person's messages in a specific channel based on the choice of aggregation



def message_aggregation(agg_choice, df):
    df_output = pd.DataFrame()
    #sort df by channel and then timestamp
    df_sorted = df.sort_values(by=['id_channel', 'timestamp'], ascending=[True, True]).reset_index(drop=True)
    channels = df_sorted['id_channel'].unique()
    print(channels)
    for channel in channels:
        channel_df = df_sorted[df_sorted['id_channel']==channel]
        channel_df = channel_df.reset_index(drop=True)
        #print(channel_df)
        users = channel_df['id_user'].unique()
        print(users)
        
        for user in users:
            user_df = channel_df[channel_df['id_user']==user]
            user_df = user_df.reset_index(drop=True)
            if agg_choice == "message":
                
                user_df.loc[:, 'Group'] = user_df.index // agg_num_messages
                df_agg = user_df.groupby('Group').agg(
                    comment=('comment', ' '.join),
                    start_timestamp=('timestamp', 'first'),
                    end_timestamp=('timestamp', 'last')
                ).reset_index(drop=True)
                df_agg.loc[:, 'User'] = user
                df_agg.loc[:, 'Channel'] = channel

                df_output = pd.concat([df_output, df_agg], ignore_index=True, axis=0)
            elif agg_choice == "time":
                # Initialize variables for tracking current aggregation
                start_time = user_df.loc[0, 'timestamp']
                current_group = 0
                aggregated_messages =[]
        
                for index, row in user_df.iterrows():
                    if row['timestamp'] > start_time + agg_time:
                    # Aggregate messages for the current group
                        aggregated_comment = ' '.join(aggregated_messages)
                        df_agg = pd.DataFrame({
                        'start_timestamp': [start_time],
                        'comment': [aggregated_comment],
                        'id_user': [user],
                        'id_channel': [channel]
                        })
                    
                        # Append to output DataFrame
                        df_output = pd.concat([df_output, df_agg], ignore_index=True)
                    
                        # Reset aggregation variables for the next group
                        start_time = row['timestamp']
                        aggregated_messages = [row['comment']]
                        current_group += 1
                    else:
                        aggregated_messages.append(row['comment'])
            
                # Final aggregation for the last group
                aggregated_comment = ' '.join(aggregated_messages)
                df_agg = pd.DataFrame({
                'start_timestamp': [start_time],
                'comment': [aggregated_comment],
                'id_user': [user],
                'id_channel': [channel]
                })
            
                # Append to output DataFrame
                df_output = pd.concat([df_output, df_agg], ignore_index=True, axis=0)
            
            elif agg_choice == "date":
                #start_date = datetime.day(user_df.loc[0, 'timestamp'])
                df_agg = user_df.groupby(user_df['timestamp'].dt.date)['comment'].apply(' '.join).reset_index()
                df_agg['id_user'] = user
                df_agg['id_channel'] = channel
            
                # Append to output DataFrame
                df_output = pd.concat([df_output, df_agg], ignore_index=True, axis=0)
                #print(df_output)
                #df_output = df_output.rename(columns={'timestamp': 'group'})
    #add an index column labelled "group" at the end
    df_output['group'] = range(1, len(df_output)+1)
    return df_output


In [128]:
test_output = message_aggregation("date", df_test)

test_output

#test_output.loc[0, 'comment']

['C019G41FR5K']
['W011D9LSALE' 'W01ATN6Q5HP' 'W011BHUE0P9' 'W012KB8C23A' 'W01966HC5QD'
 'W011889J3HV' 'W0128EBN26A' 'W011NPRL98W' 'W010TCQUDMY' 'W0115L9AF2Q'
 'W0118U7QKDH' 'W0125P5R25U' 'W011VFX2GEN' 'W011F76N1R6']


,timestamp,comment,id_user,id_channel,group
0,2020-11-09,Anyone get the rafts or are we not doing that?...,W011D9LSALE,C019G41FR5K,1
1,2020-11-10,F Did you guys do testing with more people in ...,W011D9LSALE,C019G41FR5K,2
2,2020-11-12,Does this also mean we should wear pink,W011D9LSALE,C019G41FR5K,3
3,2020-11-14,"Hey team, since we're gonna be getting wet for...",W011D9LSALE,C019G41FR5K,4
4,2020-11-16,Would it look good to have a black/dark outlin...,W011D9LSALE,C019G41FR5K,5
...,...,...,...,...,...
60,2020-11-14,"i can’t come in today, i’m sorry y’all",W0125P5R25U,C019G41FR5K,61
61,2020-11-14,Unsure but we can let you know Can anyone from...,W011VFX2GEN,C019G41FR5K,62
62,2020-11-15,"Megan, Riley and I came up with a list of rema...",W011VFX2GEN,C019G41FR5K,63
63,2020-11-16,I already have them in a PDF. Just added it to...,W011VFX2GEN,C019G41FR5K,64


In [ ]:
#try with one year (2021) of 2.009 data, write the result to a csv that I will run through LIWC

In [ ]:
#first algorithm is language style matching with the LIWC package -- I am currently exporting and re-importing from the LIWC app, to see if it works, and then will try with the API if we think it is worth paying for

#three main methods for LSM - : An early proposal for a LSM method by Niederhoffer and Pennebaker (2002), the most established LSM method of Ireland and Pennebaker (2010 (with a refinement by Müller-Frommeyer et al., 2019), and a LSM method for short online conversations by Danescu-Niculescu-Mizil et al. (2011).

#from Boyd et all 2022: ppron ipron; articles ; prep  negative  adverb auxverb  and conj 


#basic LSM implementation from Ireland and Pennebaker 2010

#run this for each time period and person combo


def LSM_basic(categories, person_a, person_b): 
    per_category = []
    for category in categories:
        C_a = person_a[category]
        C_b = person_b[category]
        lsm_c = 1 - ((math.abs(C_a - C_b)) / (C_a+C_b +0.001))
        per_category.append(lsm_c)


    lsm_tot = np.mean(per_category)
    return lsm_tot


def LSM_application(LSM_type, df): 
    categories = []


    channels = df['id_channel'].unique()
    df_result = pd.DataFrame()
    for channel in channels:
        groups = df[df['id_channel']==channel]['group'].unique()
        for group in groups:
            users= df[df['id_channel']==channel & df['group']==group]['id_user'].unique()
            for a in users:
                person_a = df[df['id_channel']==channel & df['group']==group & df['id_user'] == a]
                for b in users:
                    if b==a:
                        continue
                    person_b = df[df['id_channel']==channel & df['group']==group & df['id_user'] == b]
                    result = LSM_basic(categories, person_a, person_b)
                    df_result.concat(df_result,{'channel': channel, 'group':group, 'user_a': a, 'user_b': b, 'LSM': result})


    return df_result


In [ ]:
#another decision that we need to make is the decision of how to average the comparison across all dyads to get the overall value


#easiest way is to just average all of the dyads


def group_average():

In [ ]:
#visualization